# Task 1


In [ ]:
import random
import heapq

GRID_SIZE = 10
WALL_PROB = 0.2

# ---------------- GRID FUNCTIONS ----------------

def generate_grid():
    return [[0 if random.random() > WALL_PROB else 1 for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]

def is_valid(grid, x, y):
    return 0 <= x < GRID_SIZE and 0 <= y < GRID_SIZE and grid[x][y] == 0

def print_grid(grid, agent=None, goal=None):
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            if agent == (i, j):
                print("A", end=" ")
            elif goal == (i, j):
                print("G", end=" ")
            elif grid[i][j] == 1:
                print("#", end=" ")
            else:
                print(".", end=" ")
        print()
    print()

# ---------------- A* SEARCH ----------------

def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star(grid, start, goal):
    pq = []
    heapq.heappush(pq, (0, start))
    came_from = {}
    g_cost = {start: 0}

    while pq:
        _, current = heapq.heappop(pq)

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            return path[::-1]

        for dx, dy in [(0,1),(1,0),(0,-1),(-1,0)]:
            nx, ny = current[0]+dx, current[1]+dy
            if is_valid(grid, nx, ny):
                new_cost = g_cost[current] + 1
                if (nx, ny) not in g_cost or new_cost < g_cost[(nx, ny)]:
                    g_cost[(nx, ny)] = new_cost
                    priority = new_cost + heuristic((nx, ny), goal)
                    heapq.heappush(pq, (priority, (nx, ny)))
                    came_from[(nx, ny)] = current
    return None

# ---------------- ENVIRONMENT CHANGE ----------------

def update_environment(grid):
    for _ in range(3):
        x, y = random.randint(0,9), random.randint(0,9)
        grid[x][y] = 1 - grid[x][y]

# ---------------- ADAPTIVE AGENT ----------------

def adaptive_agent_verbose(scenario_id):
    print(f"\n========== SCENARIO {scenario_id} ==========\n")

    grid = generate_grid()
    start = (0, 0)
    goal = (9, 9)

    grid[start[0]][start[1]] = 0
    grid[goal[0]][goal[1]] = 0

    current = start
    steps = 0
    replans = 0

    print("Initial Grid:")
    print_grid(grid, current, goal)

    while current != goal:
        path = a_star(grid, current, goal)

        if not path:
            print("❌ No path available")
            return False, steps, replans

        for step in path:
            steps += 1
            print(f"Step {steps}: Moving to {step}")

            update_environment(grid)

            if not is_valid(grid, step[0], step[1]):
                replans += 1
                print("⚠ Path blocked! Replanning...\n")
                break

            current = step
            print_grid(grid, current, goal)

            if current == goal:
                print("🎯 Goal Reached!")
                return True, steps, replans

    return True, steps, replans

# ---------------- EVALUATION ----------------

def evaluate_verbose():
    success = 0
    steps_list = []
    replans_list = []

    for i in range(1, 51):
        result, steps, replans = adaptive_agent_verbose(i)

        print(f"Scenario {i} Summary:")
        print("Success:", result)
        print("Steps:", steps)
        print("Replans:", replans)
        print("-" * 40)

        if result:
            success += 1
            steps_list.append(steps)
            replans_list.append(replans)

    print("\n========== FINAL RESULTS ==========")
    print("Success Rate:", success / 50)
    print("Average Steps:", sum(steps_list) / len(steps_list))
    print("Average Replans:", sum(replans_list) / len(replans_list))

# 🔴 VERY IMPORTANT (FUNCTION CALL)
evaluate_verbose()




========== SCENARIO 1 ==========

Initial Grid:
A # . . . . . . . . 
. . . . . . # . # # 
. . . . # # . . # . 
. . . . . . . . . . 
# . . # . . . # . # 
. . . . . . . . . . 
. . . # . . . # . . 
# . . . . . . . . # 
. . . . . . # . . . 
. . . # . . . . . G 

Step 1: Moving to (1, 0)
. # . . . # # . . . 
A . . . . . # . # # 
. . . . # # . . # . 
. . . . . . . . . . 
# . . # . . . # . # 
. . . . . . . . . . 
. . . # . . . # . . 
# . . . . . . . # # 
. . . . . . # . . . 
. . . # . . . . . G 

Step 2: Moving to (1, 1)
. # . . . # # . . # 
# A . . . . # . # . 
. . . . # # . . # . 
. . . . . . . . . . 
# . . # . . . # . # 
. . . . . . . . . . 
. . . # . . . # . . 
# . . . . . . . # # 
. . . . . . # . . . 
. . . # . . . . . G 

Step 3: Moving to (1, 2)
. # . . . # # . . # 
# # A . . . # . # . 
. . . . # # . . # . 
# . . . . . . . . . 
# . . # . . . # . # 
. . . . . . . . . . 
. . . # . . . # . . 
# . . . . . . . # # 
. . . . . . # . . # 
. . . # . . . . . G 

Step 4: Moving to (1, 3)
. # . .

# Task 2

In [21]:
# ================================
# Mini Expert System (Forward Chaining)
# Cybersecurity Threat Diagnosis
# ================================

class CyberExpertSystem:
    def __init__(self):
        self.facts = {}
        self.trace = []

    # Ask user if fact is missing
    def ask(self, question):
        ans = input(question + " (yes/no): ").strip().lower()
        return ans == "yes"

    # Get fact (from user or predefined)
    def get_fact(self, fact, question):
        if fact not in self.facts:
            self.facts[fact] = self.ask(question)
        return self.facts[fact]

    # Forward chaining diagnosis
    def diagnose(self):
        self.trace.clear()

        high_traffic = self.get_fact("high_traffic", "Is network traffic unusually high?")
        user_activity = self.get_fact("user_activity", "Is there normal user activity?")
        many_ports = self.get_fact("many_ports", "Are many ports being accessed?")
        suspicious_files = self.get_fact("suspicious_files", "Are suspicious files detected?")
        antivirus_alert = self.get_fact("antivirus_alert", "Is antivirus alert triggered?")

        if high_traffic and not user_activity:
            self.trace.append(
                "IF high traffic AND no user activity THEN DDoS Attack"
            )
            return "DDoS Attack"

        if many_ports:
            self.trace.append(
                "IF many ports accessed THEN Port Scan"
            )
            return "Port Scan"

        if suspicious_files and antivirus_alert:
            self.trace.append(
                "IF suspicious files AND antivirus alert THEN Malware Infection"
            )
            return "Malware Infection"

        self.trace.append(
            "ELSE Normal Traffic"
        )
        return "Normal Traffic"


# ================================
# Reasoning Trace Printer
# ================================
def show_trace(trace):
    print("Reasoning Trace:")
    for step in trace:
        print(" -", step)


# ================================
# Test Scenarios (10 cases)
# ================================
test_cases = [
    ("DDoS Attack",  {"high_traffic": True,  "user_activity": False, "many_ports": False, "suspicious_files": False, "antivirus_alert": False}),
    ("Port Scan",   {"high_traffic": False, "user_activity": True,  "many_ports": True,  "suspicious_files": False, "antivirus_alert": False}),
    ("Malware Infection", {"high_traffic": False, "user_activity": True, "many_ports": False, "suspicious_files": True, "antivirus_alert": True}),
    ("Normal Traffic", {"high_traffic": False, "user_activity": True, "many_ports": False, "suspicious_files": False, "antivirus_alert": False}),
    ("DDoS Attack", {"high_traffic": True, "user_activity": False, "many_ports": False, "suspicious_files": False, "antivirus_alert": False}),
    ("Port Scan", {"high_traffic": False, "user_activity": True, "many_ports": True, "suspicious_files": False, "antivirus_alert": False}),
    ("Malware Infection", {"high_traffic": False, "user_activity": True, "many_ports": False, "suspicious_files": True, "antivirus_alert": True}),
    ("Normal Traffic", {"high_traffic": False, "user_activity": True, "many_ports": False, "suspicious_files": False, "antivirus_alert": False}),
    ("DDoS Attack", {"high_traffic": True, "user_activity": False, "many_ports": False, "suspicious_files": False, "antivirus_alert": False}),
    ("Normal Traffic", {"high_traffic": False, "user_activity": True, "many_ports": False, "suspicious_files": False, "antivirus_alert": False}),
]


# ================================
# Automated Testing & Accuracy
# ================================
def test_system():
    correct = 0

    for i, (expected, facts) in enumerate(test_cases, start=1):
        system = CyberExpertSystem()
        system.facts = facts  # simulated logs as facts

        result = system.diagnose()

        print(f"\nScenario {i}")
        print("Expected Diagnosis :", expected)
        print("System Diagnosis   :", result)
        show_trace(system.trace)

        if result == expected:
            correct += 1

    accuracy = (correct / len(test_cases)) * 100
    print("\n==============================")
    print("Final Accuracy:", accuracy, "%")
    print("==============================")


# ================================
# RUN TESTS
# ================================
test_system()
print("\n==============================")
print("INTERACTIVE MODE (User Input)")
print("==============================")

system = CyberExpertSystem()
result = system.diagnose()

print("\nFinal Diagnosis:", result)
show_trace(system.trace)




Scenario 1
Expected Diagnosis : DDoS Attack
System Diagnosis   : DDoS Attack
Reasoning Trace:
 - IF high traffic AND no user activity THEN DDoS Attack

Scenario 2
Expected Diagnosis : Port Scan
System Diagnosis   : Port Scan
Reasoning Trace:
 - IF many ports accessed THEN Port Scan

Scenario 3
Expected Diagnosis : Malware Infection
System Diagnosis   : Malware Infection
Reasoning Trace:
 - IF suspicious files AND antivirus alert THEN Malware Infection

Scenario 4
Expected Diagnosis : Normal Traffic
System Diagnosis   : Normal Traffic
Reasoning Trace:
 - ELSE Normal Traffic

Scenario 5
Expected Diagnosis : DDoS Attack
System Diagnosis   : DDoS Attack
Reasoning Trace:
 - IF high traffic AND no user activity THEN DDoS Attack

Scenario 6
Expected Diagnosis : Port Scan
System Diagnosis   : Port Scan
Reasoning Trace:
 - IF many ports accessed THEN Port Scan

Scenario 7
Expected Diagnosis : Malware Infection
System Diagnosis   : Malware Infection
Reasoning Trace:
 - IF suspicious files AND a

# Task 3

In [27]:
from collections import deque
import math
import random

# ---------------- GAME SETUP ---------------- #

PLAYER_X = "X"
PLAYER_O = "O"
EMPTY = " "

def initial_state():
    return [EMPTY] * 9

def available_moves(board):
    return [i for i in range(9) if board[i] == EMPTY]

def make_move(board, move, player):
    new_board = board[:]
    new_board[move] = player
    return new_board

def winner(board):
    wins = [
        (0,1,2),(3,4,5),(6,7,8),
        (0,3,6),(1,4,7),(2,5,8),
        (0,4,8),(2,4,6)
    ]
    for a,b,c in wins:
        if board[a] == board[b] == board[c] != EMPTY:
            return board[a]
    return None

def terminal(board):
    return winner(board) or EMPTY not in board

def utility(board):
    if winner(board) == PLAYER_X:
        return 1
    elif winner(board) == PLAYER_O:
        return -1
    return 0

# ---------------- MEMORY-LIMITED CACHE ---------------- #

MAX_MEMORY = 100
state_cache = deque(maxlen=MAX_MEMORY)  # FIFO memory

def store_state(board):
    state_cache.append(tuple(board))

# ---------------- MINIMAX WITH ALPHA-BETA ---------------- #

def minimax(board, depth, alpha, beta, maximizing):
    if depth == 0 or terminal(board):
        store_state(board)
        return utility(board)

    if maximizing:
        value = -math.inf
        for move in available_moves(board):
            value = max(value, minimax(
                make_move(board, move, PLAYER_X),
                depth-1, alpha, beta, False))
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return value
    else:
        value = math.inf
        for move in available_moves(board):
            value = min(value, minimax(
                make_move(board, move, PLAYER_O),
                depth-1, alpha, beta, True))
            beta = min(beta, value)
            if beta <= alpha:
                break
        return value

# ---------------- ITERATIVE DEEPENING ---------------- #

def best_move(board, max_depth):
    best_score = -math.inf
    move_choice = None

    for depth in range(1, max_depth + 1):
        for move in available_moves(board):
            score = minimax(
                make_move(board, move, PLAYER_X),
                depth, -math.inf, math.inf, False)
            if score > best_score:
                best_score = score
                move_choice = move

    return move_choice

# ---------------- BASIC OPPONENT ---------------- #

def random_opponent(board):
    return random.choice(available_moves(board))

# ---------------- GAME SIMULATION ---------------- #

def play_game(depth_limit):
    board = initial_state()

    while not terminal(board):
        # AI move
        move = best_move(board, depth_limit)
        if move is not None:
            board = make_move(board, move, PLAYER_X)

        if terminal(board):
            break

        # Opponent move
        board = make_move(board, random_opponent(board), PLAYER_O)

    return winner(board)

# ---------------- TEST RUN ---------------- #

wins = 0
games = 20

for _ in range(games):
    result = play_game(depth_limit=5)
    if result == PLAYER_X:
        wins += 1

print("Games Played:", games)
print("AI Wins:", wins)
print("Win Rate:", (wins / games) * 100, "%")
print("Memory Used:", len(state_cache))

Games Played: 20
AI Wins: 20
Win Rate: 100.0 %
Memory Used: 100


# Task 4

In [30]:
# Step 1: Import libraries
import pandas as pd
import numpy as np

# ML and preprocessing tools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, r2_score

from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression

# ------------------ Load Dataset ------------------
df = pd.read_csv("student_data.csv")

print("Dataset Shape:", df.shape)
df.head()


Dataset Shape: (395, 33)


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [31]:
# Step 2: Target variable (assuming 'G3' is final grade)
if "G3" not in df.columns:
    raise ValueError("Dataset does not have a 'G3' column. Check column names.")

y = df["G3"]
X = df.drop(columns=["G3"])

# Identify categorical and numerical columns
cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

print("Categorical columns:", list(cat_cols))
print("Numerical columns:", list(num_cols))


Categorical columns: ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']
Numerical columns: ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2']


In [32]:
# Step 3: Split dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)


Training set shape: (316, 32)
Test set shape: (79, 32)


In [33]:
# Step 4: Preprocessing
preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", StandardScaler(), num_cols)
    ]
)

# Fit preprocessing to training data (needed for Random Forest feature importance)
X_train_prep = preprocess.fit_transform(X_train)
X_test_prep = preprocess.transform(X_test)

print("Preprocessing completed.")
print("Processed feature shape:", X_train_prep.shape)


Preprocessing completed.
Processed feature shape: (316, 58)


In [34]:
# Step 5: Feature importance using Random Forest
rf_for_importance = RandomForestRegressor(n_estimators=200, random_state=42)
rf_for_importance.fit(X_train_prep, y_train)

importances = rf_for_importance.feature_importances_
feat_names = preprocess.get_feature_names_out()
feat_importance = pd.Series(importances, index=feat_names).sort_values(ascending=False)

top_features = feat_importance[:10].index.tolist()
print("Top 10 important features:\n", top_features)


Top 10 important features:
 ['num__G2', 'num__absences', 'cat__reason_home', 'num__age', 'num__G1', 'num__famrel', 'num__health', 'num__goout', 'cat__guardian_mother', 'num__studytime']


In [35]:
# Step 6: Define all models
models = {
    "RandomForest": Pipeline([("prep", preprocess),
                              ("model", RandomForestRegressor(n_estimators=200, random_state=42))]),
    "SVM": Pipeline([("prep", preprocess),
                     ("model", SVR(kernel="rbf"))]),
    "NaiveBayes": Pipeline([("prep", preprocess),
                            ("model", GaussianNB())]),
    "DecisionTree": Pipeline([("prep", preprocess),
                              ("model", DecisionTreeRegressor(random_state=42))]),
    "MLP": Pipeline([("prep", preprocess),
                     ("model", MLPRegressor(hidden_layer_sizes=(100,50), max_iter=500, random_state=42))])
}

print("Models defined:", list(models.keys()))


Models defined: ['RandomForest', 'SVM', 'NaiveBayes', 'DecisionTree', 'MLP']


In [36]:
# Step 7: Train each model and evaluate
results = {}
for name, pipe in models.items():
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    results[name] = {
        "MAE": mean_absolute_error(y_test, y_pred),
        "R2": r2_score(y_test, y_pred)
    }

print("Individual model evaluation completed.")


Individual model evaluation completed.


In [37]:
# Step 8: Hybrid model (ensemble RF + MLP predictions)
rf_preds = models["RandomForest"].predict(X_test)
mlp_preds = models["MLP"].predict(X_test)

# Use Linear Regression as meta-model
X_meta = np.vstack([rf_preds, mlp_preds]).T
meta_model = LinearRegression()
meta_model.fit(X_meta, y_test)

hybrid_pred = meta_model.predict(X_meta)
results["Hybrid"] = {
    "MAE": mean_absolute_error(y_test, hybrid_pred),
    "R2": r2_score(y_test, hybrid_pred)
}

print("Hybrid model evaluation completed.")


Hybrid model evaluation completed.


In [38]:
# Step 9: Print all model performances
print("\nModel Performance on Your Dataset:")
for m, stats in results.items():
    print(f"{m}: MAE={stats['MAE']:.3f}, R2={stats['R2']:.3f}")

# Ethical considerations
print("""
Ethical Notes:
- Predictive models should support students, not punish them.
- Ensure data privacy and avoid bias against demographics.
- Transparency and fairness must be considered before deployment.
""")



Model Performance on Your Dataset:
RandomForest: MAE=1.190, R2=0.803
SVM: MAE=1.478, R2=0.730
NaiveBayes: MAE=3.405, R2=-0.053
DecisionTree: MAE=1.278, R2=0.733
MLP: MAE=1.791, R2=0.628
Hybrid: MAE=1.204, R2=0.805

Ethical Notes:
- Predictive models should support students, not punish them.
- Ensure data privacy and avoid bias against demographics.
- Transparency and fairness must be considered before deployment.

